In [1]:
import pandas as pd
import numpy as np

In [2]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/cmc/cmc.data"

names = ["Wife's age", "Wife's education", "Husband's education", "Number of children ever born", "Wife's religion", 
"Wife's now working?", "Husband's occupation", "Standard-of-living index", "Media exposure", "Contraceptive method used"]

dataframe = pd.read_csv(data_source, names=names)

In [3]:
dataframe[:5]

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


In [5]:
dataframe.dtypes

Wife's age                         int64
Wife's education                category
Husband's education             category
Number of children ever born       int64
Wife's religion                 category
Wife's now working?             category
Husband's occupation            category
Standard-of-living index        category
Media exposure                  category
Contraceptive method used       category
dtype: object

In [6]:
for column in range(0,10):
    if column not in [0,3]:
        dataframe[names[column]] = dataframe[names[column]].astype("category")

dataframe.dtypes
# y_data = df["Contraceptive method used"].values.reshape(-1,1)

# y_data[:5]

Wife's age                         int64
Wife's education                category
Husband's education             category
Number of children ever born       int64
Wife's religion                 category
Wife's now working?             category
Husband's occupation            category
Standard-of-living index        category
Media exposure                  category
Contraceptive method used       category
dtype: object

In [7]:
y_data = dataframe.pop("Contraceptive method used")

In [8]:
x_data = pd.get_dummies(dataframe).values
y_data = pd.get_dummies(y_data).values

In [9]:
from sklearn import preprocessing as p

min_max_scaler=p.MinMaxScaler()
x_data=min_max_scaler.fit_transform(x_data)
x_data = np.insert(x_data, 0 , 1, axis=1)

In [10]:
idx=np.random.randint(y_data.shape[0],size=int(y_data.shape[0]))
training_idx = idx[:int(y_data.shape[0]*0.8)]
test_idx = idx[int(y_data.shape[0]*0.8):]

x_training, x_test = x_data[training_idx], x_data[test_idx]
y_training, y_test = y_data[training_idx], y_data[test_idx]

In [11]:
y_training.shape

(1178, 3)

In [24]:
import tensorflow as tf

x = x_training.T

W = tf.Variable(tf.random_uniform([len(y_training[0]),len(x)], -1.0, 1.0))

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

hypothesis = tf.nn.softmax(tf.matmul(W, X))

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

W_val = []
cost_val = []

a = tf.Variable(0.01)
optimizer = tf.train.GradientDescentOptimizer(a) # 라이브러리
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

y_training = y_training.T

for step in range(100000):
    sess.run(train, feed_dict={X:x, Y:y_training})
    if step % 400000 == 0:
        print (step, sess.run(cost, feed_dict={X:x, Y:y_training}), sess.run(W))

0 2855.38 [[ 0.07302348 -0.44872186  0.54661524 -0.07591087  0.08573216  0.13645442
   0.11296317 -0.7550531   0.53831768  0.37780637  0.12460548  0.35969156
   0.39586756  0.25502253 -0.11431967  0.15799445  0.26702225 -0.04632336
  -0.88839155  0.84679401 -0.25097686  0.63825774 -0.12825127 -0.78424823
   0.22607429]
 [-0.05679983  0.80839759 -0.04537201 -0.53889793 -0.87961578 -0.1465043
   0.94282097  0.26789787 -0.88790154  0.80479813  0.64363605  0.72527617
   0.31793675  0.6235736   0.09063594  0.21589944 -0.83666503  0.32845271
   0.42192024  0.6748789   0.43644649 -0.6367507  -0.48923001 -0.5019778
  -0.42635876]
 [-0.11300675 -0.74748486 -0.39433196  0.15646942  0.37180063  0.32374355
   0.39094672  0.81786931  0.80925864  0.87334681  0.73098445  0.53991604
  -0.61141503 -0.49270245  0.72333103 -0.25186092  0.29579586 -0.37144417
   0.39385024 -0.69431359  0.60349023  0.56949222 -0.53092474 -0.62151527
  -0.34476724]]


In [35]:
hypothesis_value=sess.run(hypothesis,feed_dict={X:x})
result=[np.argmax(predict) == np.argmax(original_value) for predict, original_value in zip(hypothesis_value.T,y_test)]

sum(result)/len(result)


0.32203389830508472